In [190]:
import pandas as pd

# Importing CSV Files
The CSV files are from Wireshark on attack and normal sessions that was created.

In [245]:
dfAttack = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack.csv")

In [246]:
dfAttack2 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpattack2.csv")

In [247]:
dfNormal = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal.csv")

In [248]:
dfNormal2 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal2.csv")

In [249]:
dfNormal3 = pd.read_csv(r"C:\Users\Neil Garcia\Downloads\Shared Folder - TE Machine\try\httpExclusive\httpWireshark\use\httpnormal3.csv")

--------------------------------------------------------------------------------

# Label the dataframes accordingly based on the traffic
Since the CSV files are separated, we can label the attack session with 1 and the normal session for 0. 

In [250]:
dfAttack2["Label"] = 1
dfAttack["Label"] = 1
dfNormal["Label"] = 0
dfNormal2["Label"] = 0
dfNormal3["Label"] = 0

##### Quick validation count for each rows in the dataframes
To check if the dataset is imbalance with the attack session and normal session.

In [251]:
data = [dfAttack, dfAttack2, dfNormal, dfNormal2, dfNormal3]

for i in data:
    num_rows = len(i)
    print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 1913
Number of rows in the DataFrame: 3977
Number of rows in the DataFrame: 127
Number of rows in the DataFrame: 47
Number of rows in the DataFrame: 199


------------------------------------------------------------------------

# Join the normal and attack session dataframes
Combine all dataframes into one and remove any rows that are null

In [252]:
df = pd.concat([dfAttack, dfAttack2, dfNormal, dfNormal2, dfNormal3], axis=0, ignore_index=True)

# Drop rows that are null
df.dropna(inplace=True)

In [253]:
df

,No.,Arrival Time,Time,Source,Destination,Protocol,Frame Length,Info,Full request URI,Label
0,1,"Sep 18, 2023 23:27:26.023406000 Malay Peninsul...",0.000000,192.168.95.128,192.168.95.129,HTTP,359,GET /dvwa/vulnerabilities/fi/..%2f..%2f..%2f.....,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
1,2,"Sep 18, 2023 23:27:26.326025000 Malay Peninsul...",0.302619,192.168.95.128,192.168.95.129,HTTP,302,GET /dvwa/vulnerabilities/fi/..%5cdeep_travers...,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
2,3,"Sep 18, 2023 23:27:26.628313000 Malay Peninsul...",0.604907,192.168.95.128,192.168.95.129,HTTP,253,GET /dvwa/vulnerabilities/fi/..%5c..%5cdeep_tr...,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
3,4,"Sep 18, 2023 23:27:26.931207000 Malay Peninsul...",0.907801,192.168.95.128,192.168.95.129,HTTP,333,GET /dvwa/vulnerabilities/fi/..%5c..%5c..%5cde...,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
4,5,"Sep 18, 2023 23:27:27.233910000 Malay Peninsul...",1.210504,192.168.95.128,192.168.95.129,HTTP,309,GET /dvwa/vulnerabilities/fi/..%5c..%5c..%5c.....,http://192.168.95.129/dvwa/vulnerabilities/fi/...,1
...,...,...,...,...,...,...,...,...,...,...
6258,53941,"Oct 4, 2023 14:43:23.857727013 Malay Peninsul...",347.850720,172.24.30.224,172.24.30.219,HTTP,424,GET /favicon.ico HTTP/1.1,http://172.24.30.219/favicon.ico,0
6259,54143,"Oct 4, 2023 14:43:26.622892894 Malay Peninsul...",350.615886,172.24.30.224,172.24.30.219,HTTP,549,GET /dvwa/instructions.php HTTP/1.1,http://172.24.30.219/dvwa/instructions.php,0
6260,54265,"Oct 4, 2023 14:43:28.163690396 Malay Peninsul...",352.156683,172.24.30.224,172.24.30.219,HTTP,554,GET /dvwa/instructions.php?doc=readme HTTP/1.1,http://172.24.30.219/dvwa/instructions.php?doc...,0
6261,54292,"Oct 4, 2023 14:43:28.379395600 Malay Peninsul...",352.372389,172.24.30.224,172.24.30.219,HTTP,459,GET /dvwa/favicon.ico HTTP/1.1,http://172.24.30.219/dvwa/favicon.ico,0


-------------------------------------------------

# Remove any unecessary strings in the Full request URI
Since the sessions were conducted in a controlled environment, it will be redundant as all traffic were defined. There were also strings that were unecessary that will affect the quality of dataset.

- Replace any unecessary strings
- Shuffle the rows

In [254]:
# Define the replacement patterns. Remove all unecessary wordings in the payload URI
patterns = [r'deep_traversal.txt', r'http://192.168.95.129/', r'http://172.24.30.219/']

df.loc[df['Full request URI'].str.contains('dvwa/login.php'), 'Label'] = 0

# Apply replacement patterns in a loop
for pattern in patterns:
    df['Full request URI'] = df['Full request URI'].str.replace(pattern, '', regex=True)

In [255]:
# Shuffle the rows
df = df.sample(frac=1.0, random_state=42)
print(df)

        No.                                       Arrival Time        Time  \
2304   3977  Oct  3, 2023 00:46:12.105369103 Malay Peninsul...  118.412597   
3621  17350  Oct  3, 2023 00:50:31.377152925 Malay Peninsul...  377.684381   
5224  33483  Oct  3, 2023 00:58:23.929905715 Malay Peninsul...  850.237134   
2707   8068  Oct  3, 2023 00:48:03.757847121 Malay Peninsul...  230.065075   
2596   6930  Oct  3, 2023 00:47:40.544020907 Malay Peninsul...  206.851249   
...     ...                                                ...         ...   
3772  18880  Oct  3, 2023 00:51:04.093390148 Malay Peninsul...  410.400618   
5191  33151  Oct  3, 2023 00:58:13.941151834 Malay Peninsul...  840.248380   
5226  33503  Oct  3, 2023 00:58:24.537381434 Malay Peninsul...  850.844609   
5390  35158  Oct  3, 2023 00:59:14.184783426 Malay Peninsul...  900.492011   
860     867  Sep 18, 2023 23:31:25.142526000 Malay Peninsul...  239.119120   

              Source     Destination Protocol  Frame Length  \


In [256]:
# Show all the column names of the dataframe
columns = df.columns
columns

Index(['No.', 'Arrival Time', 'Time', 'Source', 'Destination', 'Protocol',
       'Frame Length', 'Info', 'Full request URI', 'Label'],
      dtype='object')

--------------------------------

# Feature Selection
Dropping columns that are not relevant to LFI attacks or Layer 7 attacks and to the nature of the attacks as it was conducted only in a controlled environment.

In [257]:
df = df.drop(labels=['No.', 'Arrival Time', 'Time', 'Source', 'Destination', 'Protocol', 'Info', 'Frame Length'], axis=1)

----------------------

# Feature Engineering and Extraction
Using the Full request URI to help distinguish informative feature that can be used in detecting LFI attacks. Using the attack vectors that was defined in the Scope of the Study, we used methods to determine if there were attacks that behave on the following:

1. Directory Traversal
2. File Path Manipulation
3. Null Byte Poisoning
4. PHP/Data Wrapper
5. Double Encoding

### 1. Dot Dot Pwn
Check if the Full request URI has a path traversal pattern.

In [258]:
import re
    # Regular expression pattern to detect traversal sequences (e.g., ../, ../../, ??/, ?./, .?/)
traversal_pattern = r'\.\./|\?\?/|\?\./'

# Function to check for traversal sequences in a URI and return 1 if found, 0 otherwise
def has_traversal(uri):
    return 1 if re.search(traversal_pattern, uri) else 0

# Apply the has_traversal function to each row in the "Full request URI" column
df['HasDotDotPwn'] = df['Full request URI'].apply(has_traversal)

### 2. Data Wrapper Abuse
Check if these patterns are existing in the URI that may abuse the unsanitze web application throught directory or file path traversal that may result into LFI exploitation and deliberation.

In [259]:
# Keywords or patterns to detect in the URI
keywords_to_detect = ['etc/passwd', 'file:///', 'php://', 'passwd']

# Function to check for the presence of keywords in a URI and return 1 if found, 0 otherwise
def has_keywords(uri):
    for keyword in keywords_to_detect:
        if re.search(re.escape(keyword), uri):
            return 1
    return 0

# Apply the has_keywords function to each row in the "Full request URI" column
df['HasDataWrapperAbuse'] = df['Full request URI'].apply(has_keywords)

### 3. Double Encoded Sequences
Check if encoding patterns occurs on the Full request URI column since this technique is commonly used for LFI attacks.

In [260]:
# Regular expression pattern to detect encoded sequences (e.g., %2e, %2f, %5c, etc.)
encoded_sequence_pattern = r'%[0-9a-fA-F]{1}[0-9a-fA-F]*|%u[0-9a-fA-F]{4}'

# Function to check for encoded sequences in a URI and return 1 if found, 0 otherwise
def has_encoded_sequences(uri):
    return 1 if re.search(encoded_sequence_pattern, uri) else 0

# Apply the has_encoded_sequences function to each row in the "Full request URI" column
df['HasDoubleEncoding'] = df['Full request URI'].apply(has_encoded_sequences)

### 4. Domain Patterns
Determine if the URI has expressed an LFI attack. This method will count the occurence of the patterns as listed below:
- etc/passwd
- file://
- php://

In [261]:
# Keywords or patterns specific to your domain knowledge
domain_patterns = ['etc/passwd', 'file://', 'php://', 'passwd']

# Function to check for domain knowledge-based patterns in a URI
def has_domain_patterns(uri, domain_patterns):
    for pattern in domain_patterns:
        if pattern in uri:
            return 1
    return 0

# Apply the has_domain_patterns function to each row in the "URI" column
df['HasDomainPatterns'] = df['Full request URI'].apply(has_domain_patterns, domain_patterns=domain_patterns)

### 5. Null Byte Behavior
This method will check if there were null byte attacks (an LFI attack vector) present in the URI.

In [262]:
# Define a list of patterns to recognize (e.g., null byte)
patterns_to_recognize = [r'%00', r'\x00', r'0x']

# Function to check for the presence of patterns in a URI
def has_patterns(uri, patterns):
    for pattern in patterns:
        if re.search(pattern, uri):
            return 1
    return 0

# Apply the has_patterns function to each row in the "Full request URI" column
df['NullBytePattern'] = df['Full request URI'].apply(has_patterns, patterns=patterns_to_recognize)

### 6. URI Length
Counts the letter, value, and special character from Full request URI

In [263]:
# Function to count characters in a cell and return the count
def count_characters(cell_value):
    return len(str(cell_value).replace(" ",""))

# Apply the function to the "Full request URI" column and create a new "URILength" column
df['URILength'] = df['Full request URI'].apply(lambda x: count_characters(x))

### 7. URI Special Character/s
Counts the special character occurence in the Full request URI as LFI attacks are prominent in using special characters to exploit the vulnerability. 

In [264]:
# Regular expression pattern to detect special characters
special_char_pattern = r'[!@#$%^&*()_+{}\[\]:;<>,.?/~\\-]'

# Function to count special characters in a URI
def count_special_characters(uri):
    return len(re.findall(special_char_pattern, uri))

# Apply the count_special_characters function to each row in the "URI" column
df['SpecialCharCount'] = df['Full request URI'].apply(count_special_characters)

### 8. URI Path Depth
Count how many directories was traversed in the URI. THis can be referenced by counting the '/' (slash) connotation.

In [265]:
# Function to calculate the path depth of a URI
def calculate_path_depth(uri):
    # Split the URI by '/' to extract path components
    path_components = uri.split('/')
    
    # Calculate the number of directories traversed (excluding empty strings)
    path_depth = len([component for component in path_components if component and component != '..'])
    
    return path_depth

# Apply the calculate_path_depth function to each row in the "URI" column
df['PathDepth'] = df['Full request URI'].apply(calculate_path_depth)

### 9. Tokenizer 
Process to breakdown URI into chunks of strings to make meaningful components or tokens. Tokenizing URIs involves separating these components and often ignoring special characters and delimiters. It may separate it on the following:

- Scheme (protocol)
- Host 
- Path
- Query Parameter
- Fragment Identifier

Special characters are often not inclusive in this tokenization process. 

In [266]:
import nltk
# Function to tokenize a URI and count the number of tokens
def token_count(uri, tokenizer):
    tokens = tokenizer(uri)
    return len(tokens)

# Apply the token_count function to each row in the "URI" column using nltk's word tokenizer
tokenizer = nltk.word_tokenize  # You can use this for word-level tokenization
df['TokenCount'] = df['Full request URI'].apply(token_count, tokenizer=tokenizer)

### 10. Attack Vector 

In [267]:
# Check if any of the specified columns have a value (non-zero) in each row
df['HasAttackVector'] = df[['HasDotDotPwn', 'HasDataWrapperAbuse', 'HasDoubleEncoding', 'HasDomainPatterns', 'NullBytePattern']].any(axis=1).astype(int)

In [268]:
df

,Full request URI,Label,HasDotDotPwn,HasDataWrapperAbuse,HasDoubleEncoding,HasDomainPatterns,NullBytePattern,URILength,SpecialCharCount,PathDepth,TokenCount,HasAttackVector
2304,dvwa/vulnerabilities/fi/..%000x5c..%000x5cetc0...,1,0,1,1,1,1,55,9,4,7,1
3621,dvwa/vulnerabilities/fi/?.%uF025?.%uF025?.%uF0...,1,0,1,1,1,0,63,13,4,15,1
5224,dvwa/vulnerabilities/fi/%c0.%c0.%c1%9c%c0.%c0....,1,0,1,1,1,0,123,41,4,53,1
2707,dvwa/vulnerabilities/fi/.?%252f.?%252fetc%252f...,1,0,1,1,1,0,52,10,4,10,1
2596,dvwa/vulnerabilities/fi/..%01%%32%%66..%01%%32...,1,0,1,1,1,0,119,49,4,61,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3772,dvwa/vulnerabilities/fi/%5C..%c0%af%5C..%c0%af...,1,0,1,1,1,0,105,35,4,47,1
5191,dvwa/vulnerabilities/fi/%c0.%c0.%252f%c0.%c0.%...,1,0,1,1,1,0,77,19,4,21,1
5226,dvwa/vulnerabilities/fi/%c0.%c0.%c1%pc%c0.%c0....,1,0,1,1,1,0,67,17,4,21,1
5390,dvwa/vulnerabilities/fi/%252e%252e%c0%af%252e%...,1,0,1,1,1,0,103,21,4,37,1


# Save to a CSV File

In [269]:
# Use only if you want to put in a combined csv file
df.to_csv("preprocessdata.csv", index=False)